In [1]:
%pip install pandas sentence_transformers torch numpy
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os
import pickle

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        # 編碼器
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # 解碼器
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class EmbeddingDenoiser:
    def __init__(self, embedding_model_name="all-MiniLM-L6-v2", load_path=None):
        """
        初始化嵌入向量去噪模型

        參數:
            embedding_model_name: 要使用的句子嵌入模型名稱
            load_path: 若提供，將從此路徑載入預訓練的去噪模型
        """
        # 初始化句子編碼器
        self.encoder = SentenceTransformer(embedding_model_name)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()

        # 初始化自動編碼器模型
        self.model = Autoencoder(self.embedding_dim)

        # 如果提供了模型路徑，則載入模型
        if load_path and os.path.exists(load_path):
            self.load_model(load_path)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train(self, texts, noise_level=0.1, epochs=100, batch_size=32, learning_rate=0.001):
        """
        使用文本資料訓練去噪模型

        參數:
            texts: 文本列表
            noise_level: 添加的噪聲水平
            epochs: 訓練輪數
            batch_size: 批次大小
            learning_rate: 學習率
        """
        print("生成嵌入向量...")
        # 生成嵌入向量
        embeddings = np.array([self.encoder.encode(text) for text in texts])

        print("添加噪聲...")
        # 添加噪聲
        noise = np.random.normal(0, noise_level, size=embeddings.shape)
        noisy_embeddings = embeddings + noise

        # 轉換為PyTorch張量
        clean_tensor = torch.FloatTensor(embeddings).to(self.device)
        noisy_tensor = torch.FloatTensor(noisy_embeddings).to(self.device)

        # 創建資料載入器
        dataset = TensorDataset(noisy_tensor, clean_tensor)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # 定義損失函數和優化器
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        print(f"開始訓練，使用設備: {self.device}")
        # 訓練
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for data in dataloader:
                noisy_batch, clean_batch = data

                # 前向傳播
                outputs = self.model(noisy_batch)
                loss = criterion(outputs, clean_batch)
                total_loss += loss.item()

                # 反向傳播和優化
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            avg_loss = total_loss / len(dataloader)
            if (epoch+1) % 10 == 0:
                print(f'訓練進度: Epoch [{epoch+1}/{epochs}], 損失: {avg_loss:.6f}')

        print("訓練完成!")

        # 計算去噪效果
        self.model.eval()
        with torch.no_grad():
            denoised = self.model(noisy_tensor)
            final_loss = criterion(denoised, clean_tensor).item()

        print(f"最終損失: {final_loss:.6f}")
        return final_loss

    def denoise(self, texts=None, embeddings=None, noise_level=0):
        """
        對文本或嵌入向量進行去噪

        參數:
            texts: 要去噪的文本列表
            embeddings: 要去噪的嵌入向量
            noise_level: 如果大於0，先添加噪聲再去噪

        返回:
            去噪後的嵌入向量
        """
        self.model.eval()

        # 檢查輸入
        if texts is None and embeddings is None:
            raise ValueError("必須提供文本或嵌入向量")

        # 如果提供了文本，先編碼
        if embeddings is None:
            embeddings = np.array([self.encoder.encode(text) for text in texts])

        # 如果需要添加噪聲
        if noise_level > 0:
            noise = np.random.normal(0, noise_level, size=embeddings.shape)
            embeddings = embeddings + noise

        # 轉換為PyTorch張量並移到適當的設備
        input_tensor = torch.FloatTensor(embeddings).to(self.device)

        # 使用模型去噪
        with torch.no_grad():
            denoised_tensor = self.model(input_tensor)

        # 轉換回NumPy數組並返回
        return denoised_tensor.cpu().numpy()

    def save_model(self, path):
        """保存模型到指定路徑"""
        model_info = {
            'model_state_dict': self.model.state_dict(),
            'embedding_dim': self.embedding_dim,
        }

        # 確保目錄存在
        os.makedirs(os.path.dirname(path), exist_ok=True)

        # 保存模型
        torch.save(model_info, path)
        print(f"模型已保存到 {path}")

    def load_model(self, path):
        """從指定路徑載入模型"""
        if not os.path.exists(path):
            raise FileNotFoundError(f"找不到模型文件: {path}")

        # 載入模型
        model_info = torch.load(path, map_location=torch.device('cpu'))

        # 設置嵌入維度
        self.embedding_dim = model_info['embedding_dim']

        # 重新初始化模型
        self.model = Autoencoder(self.embedding_dim)

        # 載入模型權重
        self.model.load_state_dict(model_info['model_state_dict'])
        print(f"模型已從 {path} 載入")


# 使用示例
if __name__ == "__main__":
    # 1. 訓練和保存模型
    def train_and_save_model():
        # 準備訓練數據
        training_texts = [
            "Hello, how are you?",
            "I'm good, thanks for asking!",
            "What are you up to?",
            "Just reading a book.",
            "I love machine learning!",
            "Neural networks are fascinating.",
            "Have you tried this new algorithm?",
            "The weather is nice today."
        ]

        # 初始化模型
        denoiser = EmbeddingDenoiser()

        # 訓練模型
        denoiser.train(training_texts, noise_level=0.1, epochs=100)

        # 保存模型
        denoiser.save_model("models/embedding_denoiser.pt")

    # 2. 載入模型並使用
    def load_and_use_model():
        # 初始化並載入模型
        denoiser = EmbeddingDenoiser(load_path="models/embedding_denoiser.pt")

        # 準備測試數據
        test_texts = [
            "How's your day going?",
            "Machine learning is amazing!"
        ]

        # 獲取原始嵌入
        original_embeddings = np.array([denoiser.encoder.encode(text) for text in test_texts])

        # 添加噪聲
        noise_level = 0.1
        noise = np.random.normal(0, noise_level, size=original_embeddings.shape)
        noisy_embeddings = original_embeddings + noise

        # 去噪
        denoised_embeddings = denoiser.denoise(embeddings=noisy_embeddings)

        # 計算MSE
        original_vs_noisy_mse = np.mean((original_embeddings - noisy_embeddings) ** 2)
        original_vs_denoised_mse = np.mean((original_embeddings - denoised_embeddings) ** 2)

        print(f"測試結果:")
        print(f"原始 vs 有噪聲 MSE: {original_vs_noisy_mse:.6f}")
        print(f"原始 vs 去噪後 MSE: {original_vs_denoised_mse:.6f}")
        print(f"改善幅度: {original_vs_noisy_mse - original_vs_denoised_mse:.6f}")

    # 3. 實際使用案例
    def real_world_example():
        # 載入模型
        denoiser = EmbeddingDenoiser(load_path="models/embedding_denoiser.pt")

        # 假設我們有一些來自資料庫或外部API的嵌入向量
        # 這些向量可能因為各種原因含有噪聲
        texts = [
            "Can you help me with this problem?",
            "I need assistance with my project."
        ]

        # 編碼並去噪
        clean_embeddings = denoiser.denoise(texts=texts)

        print(f"已處理 {len(texts)} 個文本嵌入向量")
        print(f"去噪後的嵌入向量形狀: {clean_embeddings.shape}")

        # 現在可以使用這些去噪後的向量進行下游任務
        # 例如，計算相似度
        from sklearn.metrics.pairwise import cosine_similarity
        similarity = cosine_similarity([clean_embeddings[0]], [clean_embeddings[1]])[0][0]
        print(f"兩個文本的相似度: {similarity:.4f}")

    # 取消下方註釋來運行不同的範例
    # train_and_save_model()
    # load_and_use_model()
    # real_world_example()

    # 或者執行完整流程
    print("執行完整流程示例")
    train_and_save_model()
    load_and_use_model()
    real_world_example()

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB ? eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 9.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.8/11.5 MB 29.9 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.5 MB 26.1 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.5 MB 43.3 MB/s eta 0:00:01
   ---------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


執行完整流程示例


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


生成嵌入向量...
添加噪聲...
開始訓練，使用設備: cpu
訓練進度: Epoch [10/100], 損失: 0.002642
訓練進度: Epoch [20/100], 損失: 0.001656
訓練進度: Epoch [30/100], 損失: 0.001179
訓練進度: Epoch [40/100], 損失: 0.000684
訓練進度: Epoch [50/100], 損失: 0.000329
訓練進度: Epoch [60/100], 損失: 0.000124
訓練進度: Epoch [70/100], 損失: 0.000035
訓練進度: Epoch [80/100], 損失: 0.000009
訓練進度: Epoch [90/100], 損失: 0.000003
訓練進度: Epoch [100/100], 損失: 0.000001
訓練完成!
最終損失: 0.000001
模型已保存到 models/embedding_denoiser.pt
模型已從 models/embedding_denoiser.pt 載入
測試結果:
原始 vs 有噪聲 MSE: 0.010031
原始 vs 去噪後 MSE: 0.001381
改善幅度: 0.008651
模型已從 models/embedding_denoiser.pt 載入
已處理 2 個文本嵌入向量
去噪後的嵌入向量形狀: (2, 384)
兩個文本的相似度: 0.9549


In [ ]:
a = int(input())
b = a + 1
print(b)
# 輸出剛剛輸入的數字+1


13
14


In [ ]:
%pip install fastapi uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import numpy as np
import torch
import os
from sentence_transformers import SentenceTransformer

# 匯入之前定義的 Autoencoder 和 EmbeddingDenoiser 類
class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        # 編碼器
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU()
        )
        # 解碼器
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class EmbeddingDenoiser:
    def __init__(self, embedding_model_name="all-MiniLM-L6-v2", load_path=None):
        # 初始化句子編碼器
        self.encoder = SentenceTransformer(embedding_model_name)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()

        # 初始化自動編碼器模型
        self.model = Autoencoder(self.embedding_dim)

        # 如果提供了模型路徑，則載入模型
        if load_path and os.path.exists(load_path):
            self.load_model(load_path)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def denoise(self, texts=None, embeddings=None, noise_level=0):
        self.model.eval()

        # 檢查輸入
        if texts is None and embeddings is None:
            raise ValueError("必須提供文本或嵌入向量")

        # 如果提供了文本，先編碼
        if embeddings is None:
            embeddings = np.array([self.encoder.encode(text) for text in texts])

        # 如果需要添加噪聲
        if noise_level > 0:
            noise = np.random.normal(0, noise_level, size=embeddings.shape)
            embeddings = embeddings + noise

        # 轉換為PyTorch張量並移到適當的設備
        input_tensor = torch.FloatTensor(embeddings).to(self.device)

        # 使用模型去噪
        with torch.no_grad():
            denoised_tensor = self.model(input_tensor)

        # 轉換回NumPy數組並返回
        return denoised_tensor.cpu().numpy()

    def load_model(self, path):
        """從指定路徑載入模型"""
        if not os.path.exists(path):
            raise FileNotFoundError(f"找不到模型文件: {path}")

        # 載入模型
        model_info = torch.load(path, map_location=torch.device('cpu'))

        # 設置嵌入維度
        self.embedding_dim = model_info['embedding_dim']

        # 重新初始化模型
        self.model = Autoencoder(self.embedding_dim)

        # 載入模型權重
        self.model.load_state_dict(model_info['model_state_dict'])
        print(f"模型已從 {path} 載入")

# FastAPI 請求與回應模型
class TextDenoiseRequest(BaseModel):
    texts: list[str] = Field(..., description="要去噪的文本列表")
    noise_level: float = Field(0.0, description="可選參數，如果大於0，先添加噪聲再去噪", ge=0.0)

class EmbeddingDenoiseRequest(BaseModel):
    embeddings: list[list[float]] = Field(..., description="要去噪的嵌入向量列表")
    noise_level: float = Field(0.0, description="可選參數，如果大於0，先添加噪聲再去噪", ge=0.0)

class DenoiseResponse(BaseModel):
    denoised_embeddings: list[list[float]] = Field(..., description="去噪後的嵌入向量")
    shape: list[int] = Field(..., description="去噪後嵌入向量的形狀")
    processing_time_ms: float = Field(..., description="處理時間（毫秒）")

# 初始化 FastAPI 應用
app = FastAPI(
    title="嵌入向量去噪 API",
    description="使用自動編碼器對文本嵌入向量進行去噪",
    version="1.0.0"
)

# 全局變數，存儲模型實例
denoiser = None

@app.on_event("startup")
async def load_denoiser_model():
    """在應用啟動時載入模型"""
    global denoiser
    model_path = os.getenv("MODEL_PATH", "models/embedding_denoiser.pt")

    try:
        # 初始化去噪器並載入模型
        denoiser = EmbeddingDenoiser(load_path=model_path)
        print(f"已載入去噪模型，設備: {denoiser.device}")
    except Exception as e:
        print(f"載入模型失敗: {e}")
        # 如果沒有找到模型，可以選擇訓練一個
        if not os.path.exists(model_path):
            print("模型不存在，正在訓練新模型...")
            # 這裡可以加入簡單的訓練代碼
            # 或者只報錯
            raise Exception("模型不存在，請先訓練模型")

@app.get("/")
async def root():
    """API根路徑，返回簡單的歡迎信息"""
    return {"message": "歡迎使用嵌入向量去噪 API"}

@app.post("/denoise/text", response_model=DenoiseResponse)
async def denoise_text(request: TextDenoiseRequest):
    """
    基於文本的去噪API

    將文本編碼為嵌入向量，然後進行去噪處理
    """
    global denoiser

    if denoiser is None:
        raise HTTPException(status_code=503, detail="模型尚未載入，請稍後再試")

    try:
        import time
        start_time = time.time()

        # 處理文本
        denoised = denoiser.denoise(
            texts=request.texts,
            noise_level=request.noise_level
        )

        processing_time = (time.time() - start_time) * 1000  # 轉為毫秒

        # 轉換為標準Python類型以進行JSON序列化
        denoised_list = denoised.tolist()

        return {
            "denoised_embeddings": denoised_list,
            "shape": list(denoised.shape),
            "processing_time_ms": processing_time
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"處理失敗: {str(e)}")

@app.post("/denoise/embedding", response_model=DenoiseResponse)
async def denoise_embedding(request: EmbeddingDenoiseRequest):
    """
    基於嵌入向量的去噪API

    直接處理提供的嵌入向量，進行去噪處理
    """
    global denoiser

    if denoiser is None:
        raise HTTPException(status_code=503, detail="模型尚未載入，請稍後再試")

    import time
    start_time = time.time()

    # 將列表轉換為NumPy數組
    embeddings_array = np.array(request.embeddings, dtype=np.float32)

    # 處理嵌入向量
    denoised = denoiser.denoise(
        embeddings=embeddings_array,
        noise_level=request.noise_level
    )

    processing_time = (time.time() - start_time) * 1000  # 轉為毫秒

    # 轉換為標準Python類型以進行JSON序列化
    denoised_list = denoised.tolist()

    return "denoised_embeddings"


SyntaxError: invalid syntax (<ipython-input-6-de04fb8c3aaf>, line 1)

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union
import numpy as np
import torch
import os
import uvicorn
from sentence_transformers import SentenceTransformer

# 匯入之前定義的 Autoencoder 和 EmbeddingDenoiser 類
class Autoencoder(torch.nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        # 編碼器
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU()
        )
        # 解碼器
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class EmbeddingDenoiser:
    def __init__(self, embedding_model_name="all-MiniLM-L6-v2", load_path=None):
        # 初始化句子編碼器
        self.encoder = SentenceTransformer(embedding_model_name)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()

        # 初始化自動編碼器模型
        self.model = Autoencoder(self.embedding_dim)

        # 如果提供了模型路徑，則載入模型
        if load_path and os.path.exists(load_path):
            self.load_model(load_path)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def denoise(self, texts=None, embeddings=None, noise_level=0):
        self.model.eval()

        # 檢查輸入
        if texts is None and embeddings is None:
            raise ValueError("必須提供文本或嵌入向量")

        # 如果提供了文本，先編碼
        if embeddings is None:
            embeddings = np.array([self.encoder.encode(text) for text in texts])

        # 如果需要添加噪聲
        if noise_level > 0:
            noise = np.random.normal(0, noise_level, size=embeddings.shape)
            embeddings = embeddings + noise

        # 轉換為PyTorch張量並移到適當的設備
        input_tensor = torch.FloatTensor(embeddings).to(self.device)

        # 使用模型去噪
        with torch.no_grad():
            denoised_tensor = self.model(input_tensor)

        # 轉換回NumPy數組並返回
        return denoised_tensor.cpu().numpy()

    def load_model(self, path):
        """從指定路徑載入模型"""
        if not os.path.exists(path):
            raise FileNotFoundError(f"找不到模型文件: {path}")

        # 載入模型
        model_info = torch.load(path, map_location=torch.device('cpu'))

        # 設置嵌入維度
        self.embedding_dim = model_info['embedding_dim']

        # 重新初始化模型
        self.model = Autoencoder(self.embedding_dim)

        # 載入模型權重
        self.model.load_state_dict(model_info['model_state_dict'])
        print(f"模型已從 {path} 載入")

# FastAPI 請求與回應模型
class TextDenoiseRequest(BaseModel):
    texts: List[str] = Field(..., description="要去噪的文本列表")
    noise_level: float = Field(0.0, description="可選參數，如果大於0，先添加噪聲再去噪", ge=0.0)

class EmbeddingDenoiseRequest(BaseModel):
    embeddings: List[List[float]] = Field(..., description="要去噪的嵌入向量列表")
    noise_level: float = Field(0.0, description="可選參數，如果大於0，先添加噪聲再去噪", ge=0.0)

class DenoiseResponse(BaseModel):
    denoised_embeddings: List[List[float]] = Field(..., description="去噪後的嵌入向量")
    shape: List[int] = Field(..., description="去噪後嵌入向量的形狀")
    processing_time_ms: float = Field(..., description="處理時間（毫秒）")

# 初始化 FastAPI 應用
app = FastAPI(
    title="嵌入向量去噪 API",
    description="使用自動編碼器對文本嵌入向量進行去噪",
    version="1.0.0"
)

# 全局變數，存儲模型實例
denoiser = None

@app.on_event("startup")
async def load_denoiser_model():
    """在應用啟動時載入模型"""
    global denoiser
    model_path = os.getenv("MODEL_PATH", "models/embedding_denoiser.pt")

    try:
        # 初始化去噪器並載入模型
        denoiser = EmbeddingDenoiser(load_path=model_path)
        print(f"已載入去噪模型，設備: {denoiser.device}")
    except Exception as e:
        print(f"載入模型失敗: {e}")
        # 如果沒有找到模型，可以選擇訓練一個
        if not os.path.exists(model_path):
            print("模型不存在，正在訓練新模型...")
            # 這裡可以加入簡單的訓練代碼
            # 或者只報錯
            raise Exception("模型不存在，請先訓練模型")

@app.get("/")
async def root():
    """API根路徑，返回簡單的歡迎信息"""
    return {"message": "歡迎使用嵌入向量去噪 API"}

@app.post("/denoise/text", response_model=DenoiseResponse)
async def denoise_text(request: TextDenoiseRequest):
    """
    基於文本的去噪API

    將文本編碼為嵌入向量，然後進行去噪處理
    """
    global denoiser

    if denoiser is None:
        raise HTTPException(status_code=503, detail="模型尚未載入，請稍後再試")

    try:
        import time
        start_time = time.time()

        # 處理文本
        denoised = denoiser.denoise(
            texts=request.texts,
            noise_level=request.noise_level
        )

        processing_time = (time.time() - start_time) * 1000  # 轉為毫秒

        # 轉換為標準Python類型以進行JSON序列化
        denoised_list = denoised.tolist()

        return {
            "denoised_embeddings": denoised_list,
            "shape": list(denoised.shape),
            "processing_time_ms": processing_time
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"處理失敗: {str(e)}")

@app.post("/denoise/embedding", response_model=DenoiseResponse)
async def denoise_embedding(request: EmbeddingDenoiseRequest):
    """
    基於嵌入向量的去噪API

    直接處理提供的嵌入向量，進行去噪處理
    """
    global denoiser

    if denoiser is None:
        raise HTTPException(status_code=503, detail="模型尚未載入，請稍後再試")

    try:
        import time
        start_time = time.time()

        # 將列表轉換為NumPy數組
        embeddings_array = np.array(request.embeddings, dtype=np.float32)

        # 處理嵌入向量
        denoised = denoiser.denoise(
            embeddings=embeddings_array,
            noise_level=request.noise_level
        )

        processing_time = (time.time() - start_time) * 1000  # 轉為毫秒

        # 轉換為標準Python類型以進行JSON序列化
        denoised_list = denoised.tolist()

        return {
            "denoised_embeddings": denoised_list,
            "shape": list(denoised.shape),
            "processing_time_ms": processing_time
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"處理失敗: {str(e)}")

@app.get("/health")
async def health_check():
    """健康檢查端點"""
    global denoiser
    return {
        "status": "healthy" if denoiser is not None else "not_ready",
        "model_loaded": denoiser is not None,
        "device": str(denoiser.device) if denoiser is not None else None
    }

if __name__ == "__main__":
    # 設置端口
    port = int(os.getenv("PORT", 8000))

    # 啟動服務器
    uvicorn.run("app:app", host="0.0.0.0", port=port, reload=True)

In [ ]:
%pip install fastapi uvicorn pyngrok nest-asyncio python-multipart
from fastapi import FastAPI, UploadFile, File
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import random

# 避免 async loop 衝突
nest_asyncio.apply()

# 初始化 FastAPI app
app = FastAPI()

# 模擬訓練：讀檔後回傳假損失值
@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    content = await file.read()
    text = content.decode("utf-8")

    # 模擬訓練邏輯
    fake_loss = round(random.uniform(0.01, 1.0), 4)
    file_len = len(text.split())

    return {
        "filename": file.filename,
        "word_count": file_len,
        "loss": fake_loss
    }

# 設定 ngrok authtoken
# 將 "YOUR_AUTHTOKEN" 替換成你自己的 authtoken
ngrok.set_auth_token("YOUR_AUTHTOKEN")

# 開 ngrok 隧道
public_url = ngrok.connect(8000)
print(f"FastAPI 公開網址: {public_url}/docs")

# 啟動 server（背景）
uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:pyngrok.process.ngrok:t=2025-04-15T02:44:40+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.